In [1]:
from math import ceil
import copy
from collections import defaultdict

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np
import numpy.linalg as nla

from sim_lib.simulation import run_simulation
from sim_lib.graph_create import kleinberg_grid, reduce_providers_simplest, powerlaw_dist_time
from sim_lib.sim_strategies import EvenAlloc, MWU
import sim_lib.util as util

init_notebook_mode(connected=True)

In [2]:
# Constants
gl = 10 # grid length
regularity = 4

# MWU parameters
lrate = 0.5

In [3]:
def create_kg(p, r):
    return kleinberg_grid(gl, gl, r, p)

In [4]:
def plot_util_comp(given_utils, r, diam):
    fig = go.Figure()
    
    social_utils, even_utils = given_utils

    #Add averages trace
    p_vals = [ p for (p, uts) in social_utils ]
    ut_vals = [ np.average(uts) for (p, uts) in social_utils ]
    fig.add_trace(go.Scatter(x=p_vals,
        y=ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare rate (MWU)', marker=dict(color='Blue')))
    
    even_p_vals = [ p for (p, uts) in even_utils ]
    even_ut_vals = [ np.average(uts) for (p, uts) in even_utils ]
    fig.add_trace(go.Scatter(x=even_p_vals,
        y=even_ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare rate (even alloc)',
        marker=dict(color='Orange')))
    
    #Add individual points trace
    indiv_p_vals = []
    indiv_ut_vals = []
    for (p, uts) in social_utils:
        indiv_ut_vals.extend(uts)
        indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=indiv_p_vals,
        y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='LightSkyBlue'), name='social welfare per p'))

    even_indiv_p_vals = []
    even_indiv_ut_vals = []
    for (p, uts) in even_utils:
        even_indiv_ut_vals.extend(uts)
        even_indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=even_indiv_p_vals,
        y=even_indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='rgba(244,204,102,1)'),
        name='social welfare per p (even alloc)'))


    fig.layout.update(showlegend=False)
    title_fmt = "Average Social Welfare (Normalized) vs p (r={0}, diam={1})"
    plot_title = title_fmt.format(r, diam)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='avg social welfare (MWU)'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [5]:
def plot_dist_ratios(dratio_means, r, diam):
    fig = go.Figure()
    
    dist_names = ['MWU', 'Even']
    lcolors = ['Blue', 'darkorange']
    mcolors = ['LightSkyBlue', 'orange']
    
    for idx, dratio_vals in enumerate(dratio_means):
        name = dist_names[idx]
        lcol = lcolors[idx]
        mcol = mcolors[idx]
        
        #Add averages trace
        p_vals = [ p for (p, dratio) in dratio_vals ]
        den_avgs = [ np.average(dratio) for (p, dratio) in dratio_vals ]
        fig.add_trace(go.Scatter(x=p_vals,
            y=den_avgs, mode='lines+markers', hoverinfo='skip',
            name=f'avg {name} distance ratio', marker=dict(color=lcol)))

        #Add individual points trace
        indiv_p_vals = []
        indiv_den_vals = []
        for (p, dratio) in dratio_vals:
            indiv_den_vals.extend(dratio)
            indiv_p_vals.extend([p] * len(dratio))
        fig.add_trace(go.Scatter(x=indiv_p_vals,
            y=indiv_den_vals, mode='markers', hoverinfo='skip', opacity=0.7,
            marker=dict(color=mcol), name=f'{name} distance ratio'))

    fig.layout.update(showlegend=False)
    title_fmt = "Decentralized Search Distance Optimality Ratio vs p (r={0}, diam={1})"
    plot_title = title_fmt.format(r, diam)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='Dist Avg Ratio'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [6]:
def comp_strat(graph_func, strat, strat_params, plaw_resources=False, simplest=False):
    for r in [ 2 ** k for k in range(1, 8) ][::-1]:
        social_utils = []
        even_utils = []
        
        mwu_ratios = []
        even_ratios = []
        
        for p in np.linspace(1, 0, 10, endpoint=False):
            utils = []
            eutils = []
            
            mwu_ratio_means = []
            even_ratio_means = []
            
            num_iter = 10
            for i in range(num_iter):
                G = graph_func(p, r)

                graph_diam = util.calc_diameter(G)

                if simplest:
                    reduce_providers_simplest(G)
                
                #Get opt to find shortest paths later on
                opt_seeds = util.opt_vertices(G)
                initial_utils = set([ v.utility for v in G.vertices ])
                initial_seeds = { ut : [ v for v in G.vertices if v == ut ] 
                                 for ut in initial_utils }
                
                if plaw_resources:
                    powerlaw_dist_time(G, 2)
                    
                #Make copy of graph to run even alloc strategy on
                G_even = copy.deepcopy(G)

                #Initialize strategy
                sim_strat = strat(**strat_params)
                sim_strat.initialize_model(G)

                sim_g, sim_utils, mwu_ut_map = run_simulation(G, sim_strat, True)
                
                #Initialize even alloc strategy
                even_strat = EvenAlloc()
                even_strat.initialize_model(G_even)
                
                sim_g_even, sim_utils_even, even_ut_map = run_simulation(G_even, even_strat, True)

                #Get global social welfare at end of simulation normalized by size
                utils.append(sum(sim_utils[-1]) / len(G.vertices))
                eutils.append(sum(sim_utils_even[-1]) / len(G_even.vertices))
                
                #Check decentralized search success
                unif_weight = defaultdict(lambda : defaultdict(lambda : 1))
                
                #Get inverse normalized weights
                mwu_weight = {}
                for vtx in G.vertices:
                    mwu_weight[vtx] = {}
                    tot_weight = sum(sim_strat.vtx_weights[vtx].values())
                    for nbor in vtx.nbors:
                        mwu_weight[vtx][nbor] = 1 - (sim_strat.vtx_weights[vtx][nbor] / tot_weight)
                
                true_apsp, ta_nxt = util.weighted_apsp(G, unif_weight)
#                mwu_apsp, ma_nxt = util.weighted_apsp(G, mwu_weight)
#                mwu_el_apsp = util.fw_edge_len(G, ma_nxt)
                
                #Get error for closest starting opt vertices
                mwu_search_ratios = []
                even_search_ratios = []
                
                def vtx_search_ratio(vtx, sim_ut_map):
                    num_hops_final = sim_ut_map[vtx]['iter']
                    final_ut_source = sim_ut_map[vtx]['from']
                    final_ut = sim_ut_map[vtx]['ut']
                    
                    closest_same_ut_dist = num_hops_final
                    closest_same_ut_seed = final_ut_source
                    for utx in initial_seeds[final_ut]:
                        if num_hops_final == 0:
                            break
                        if true_apsp[vtx.vnum][utx.vnum] < closest_same_ut_dist:
                            closest_same_ut_dist = true_apsp[vtx.vnum][utx.vnum]
                            closest_same_ut_seed = utx
                    
                    # Get relative error of num iters to final vs shortest path to closest vtx giving final
                    if closest_same_ut_dist == 0:
                        if num_hops_final == 0:
                            return final_ut
                        else:
                            raise ValueError('Received starting utility from someone other than self')
                            
                    return num_hops_final / closest_same_ut_dist
                
                for vtx in G.vertices:
                    mwu_search_ratios.append(vtx_search_ratio(vtx, mwu_ut_map))
                    
                for vtx in G_even.vertices:
                    even_search_ratios.append(vtx_search_ratio(vtx, even_ut_map))
                    
                mwu_mean = np.mean(np.array(mwu_search_ratios))
                mwu_ratio_means.append(mwu_mean)
                
                even_mean = np.mean(np.array(even_search_ratios))
                even_ratio_means.append(even_mean)
                
            social_utils.append((p, utils))
            even_utils.append((p, eutils))
            
            mwu_ratios.append((p, mwu_ratio_means))
            even_ratios.append((p, even_ratio_means))

        plot_util_comp((social_utils, even_utils), r, graph_diam)
        
        plot_dist_ratios((mwu_ratios, even_ratios), r, graph_diam)

## Evenly distributed resources

### One optimal provider

In [7]:
#comp_strat(create_kg, MWU, {'lrate' : lrate}, simplest=True)

### Independently distributed providers

In [8]:
comp_strat(create_kg, MWU, {'lrate' : lrate}, simplest=False)

## Powerlaw distributed resources

### One optimal provider

In [9]:
#comp_strat(create_kg, MWU, {'lrate' : lrate}, plaw_resources=True, simplest=True)

### Independently distributed providers

In [10]:
comp_strat(create_kg, MWU, {'lrate' : lrate}, plaw_resources=True, simplest=False)